In [1]:
import pandas as pd
import numpy as np
import shapely
import matplotlib.pylab as plt
import geopandas as gpd
import fiona
import rtree

from geopandas import GeoDataFrame
from fiona.crs import from_epsg
from shapely.geometry import Point

%matplotlib inline

In [2]:
path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/Cleaned_DataSets/"
q_score = pd.read_csv(path + "streetscore_newyorkcity.csv")
q_score.head()

,latitude,longitude,q-score
0,40.700909,-74.013504,11.062166
1,40.701000,-74.013878,10.818611
2,40.701080,-74.012878,12.677955
3,40.701187,-74.013268,11.417325
4,40.701244,-74.012115,25.199091


In [3]:
crs = None
geometry = [Point(xy) for xy in zip(q_score.longitude, q_score.latitude)]
qscore_geo = GeoDataFrame(q_score, crs=crs, geometry=geometry)
qscore_geo.crs = fiona.crs.from_epsg(4326)

In [4]:
qscore_geo.head()

,latitude,longitude,q-score,geometry
0,40.700909,-74.013504,11.062166,POINT (-74.013504 40.700909)
1,40.701000,-74.013878,10.818611,POINT (-74.01387800000001 40.701)
2,40.701080,-74.012878,12.677955,POINT (-74.012878 40.70108)
3,40.701187,-74.013268,11.417325,POINT (-74.01326800000001 40.701187)
4,40.701244,-74.012115,25.199091,POINT (-74.01211500000001 40.701244)


In [5]:
shape_path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/bx_mappluto_16v2/BXMapPLUTO.shp"
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.to_crs(epsg=4326, inplace=True)

In [6]:
ct_shape.head()

,APPBBL,APPDate,Address,AreaSource,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BldgDepth,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,None,270 EAST 137 STREET,2,138600.0,831150.0,2.023180e+09,11700,K7,100.0,...,0,1911,None,10454,M1-4/R7A,None,None,None,6a,POLYGON ((-73.92806702357275 40.80998925624725...
1,0.0,None,69 BRUCKNER BOULEVARD,2,24211.0,35042.0,2.022960e+09,5120,S3,64.0,...,0,1915,None,10454,M1-2/R6A,None,None,None,6a,POLYGON ((-73.92619928804308 40.80644360137782...
2,0.0,None,348 EAST 134 STREET,2,18000.0,188100.0,2.022960e+09,4875,E9,85.0,...,0,2008,None,10454,M1-2/R6A,None,None,None,6a,POLYGON ((-73.92624372459983 40.80710035850777...
3,0.0,None,51 BRUCKNER BOULEVARD,2,10203.0,13989.0,2.022960e+09,5120,S4,64.0,...,0,1915,None,10454,M1-2/R6A,None,None,None,6a,"POLYGON ((-73.92659470483831 40.8069296690564,..."
4,0.0,None,2441 3 AVENUE,2,315450.0,1294650.0,2.023190e+09,83936,D9,135.0,...,0,1917,None,10451,M1-3/R8,None,None,None,6a,POLYGON ((-73.93065658344733 40.80937264330601...


In [7]:
ct_shape.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AreaSource',
       u'AssessLand',  u'AssessTot',        u'BBL',   u'BldgArea',
        u'BldgClass',  u'BldgDepth',  u'BldgFront',      u'Block',
         u'BoroCode',    u'Borough',   u'BsmtCode',   u'BuiltFAR',
           u'CB2010',         u'CD',     u'CT2010',    u'ComArea',
          u'CommFAR',    u'CondoNo',    u'Council',  u'EDesigNum',
        u'Easements', u'ExemptLand',  u'ExemptTot',        u'Ext',
         u'FacilFAR', u'FactryArea',   u'FireComp', u'GarageArea',
       u'HealthArea',   u'HistDist', u'IrrLotCode',    u'LandUse',
         u'Landmark',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType',  u'LtdHeight', u'MAPPLUTO_F',
         u'NumBldgs',  u'NumFloors', u'OfficeArea',  u'OtherArea',
         u'Overlay1',   u'Overlay2',  u'OwnerName',  u'OwnerType',
       u'PLUTOMapID', u'PolicePrct',   u'ProxCode',    u'ResArea',
         u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Le

In [8]:
BX_Shape = ct_shape.drop([u'Borough', u'CD', u'CB2010', u'Council', u'SanitBoro', u'SanitDistr', u'SanitSub', 
                          u'Overlay1',u'Overlay2', u'SPDist1', u'SPDist2', u'SPDist3', u'LtdHeight', u'SplitZone', 
                          u'Easements', u'OwnerName', u'AreaSource', u'IrrLotCode', u'BsmtCode', u'AssessLand', 
                          u'AssessTot', u'ExemptLand', u'ExemptTot', u'YearAlter1', u'YearAlter2',u'BuiltFAR', 
                          u'ResidFAR', u'CommFAR', u'FacilFAR', u'BoroCode', u'BBL', u'CondoNo', u'Tract2010', 
                          u'ZoneMap', u'ZMCode', u'Sanborn', u'TaxMap', u'EDesigNum', u'APPBBL', u'APPDate', 
                          u'PLUTOMapID', u'Version', u'Ext', u'HistDist', u'Landmark', u'ZoneDist2', u'ZoneDist3', 
                          u'ZoneDist4'], axis = 1)

In [9]:
BX_Shape.columns

Index([   u'Address',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',     u'CT2010',    u'ComArea',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
          u'LandUse',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',  u'OwnerType',
       u'PolicePrct',   u'ProxCode',    u'ResArea', u'RetailArea',
       u'SHAPE_Area', u'SHAPE_Leng', u'SchoolDist',  u'StrgeArea',
         u'UnitsRes', u'UnitsTotal',     u'XCoord',     u'YCoord',
        u'YearBuilt',    u'ZipCode',  u'ZoneDist1',   u'geometry'],
      dtype='object')

In [10]:
BX_Shape_2 = BX_Shape.drop([u'Address', u'BldgClass', u'BldgDepth', u'ComArea', u'FactryArea',
              u'FireComp', u'GarageArea', u'HealthArea', u'Lot',u'LotDepth',u'LotFront',
              u'LotType',u'OfficeArea',  u'OtherArea',  u'OwnerType',u'PolicePrct',   
              u'ProxCode',    u'ResArea', u'RetailArea',u'SHAPE_Leng', u'SchoolDist',  
              u'StrgeArea', u'UnitsRes',u'ZoneDist1'],axis = 1)
BX_Shape_2.columns

Index([  u'BldgArea',  u'BldgFront',      u'Block',     u'CT2010',
          u'LandUse',    u'LotArea', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'SHAPE_Area', u'UnitsTotal',     u'XCoord',
           u'YCoord',  u'YearBuilt',    u'ZipCode',   u'geometry'],
      dtype='object')

In [11]:
BX_Shape_2.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry
0,11700,39.00,2318,51,05,5139,0,1,3.0,5994.020237,3,1004162,234443,1911,10454,POLYGON ((-73.92806702357275 40.80998925624725...
1,5120,20.00,2296,19,04,2000,0,1,4.0,2035.046764,4,1004696,233144,1915,10454,POLYGON ((-73.92619928804308 40.80644360137782...
2,4875,25.00,2296,19,06,2500,0,1,1.0,2432.957975,1,1004633,233295,2008,10454,POLYGON ((-73.92624372459983 40.80710035850777...
3,5120,20.00,2296,19,04,2000,0,1,4.0,2111.489026,5,1004537,233231,1915,10454,"POLYGON ((-73.92659470483831 40.8069296690564,..."
4,83936,71.25,2319,51,03,35049,0,2,5.0,35247.169036,50,1003568,234178,1917,10451,POLYGON ((-73.93065658344733 40.80937264330601...


In [12]:
BX_Shape_joined = gpd.sjoin(BX_Shape_2, qscore_geo, how="inner", op='intersects')
BX_Shape_joined.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry,index_right,latitude,longitude,q-score
4,83936,71.25,2319,51,03,35049,0,2,5.0,35247.169036,50,1003568,234178,1917,10451,POLYGON ((-73.93065658344733 40.80937264330601...,22893,40.809277,-73.930168,20.231396
4,83936,71.25,2319,51,03,35049,0,2,5.0,35247.169036,50,1003568,234178,1917,10451,POLYGON ((-73.93065658344733 40.80937264330601...,23052,40.809601,-73.930351,22.881989
5,0,0.00,2295,19,11,13654,0,0,0.0,14131.400385,0,0,0,0,10454,POLYGON ((-73.92655004076623 40.80595427345522...,21433,40.806026,-73.926788,8.982685
6,0,0.00,2309,19,10,42043,0,0,0.0,43106.622601,0,1004073,233573,0,10454,POLYGON ((-73.92776338651244 40.80775127262513...,22142,40.807652,-73.928101,25.120745
6,0,0.00,2309,19,10,42043,0,0,0.0,43106.622601,0,1004073,233573,0,10454,POLYGON ((-73.92776338651244 40.80775127262513...,22143,40.807652,-73.928474,30.488537


In [13]:
BX_Shape_joined_2 = gpd.sjoin(qscore_geo, BX_Shape_2, how="inner", op='intersects')
BX_Shape_joined_2.head()

,latitude,longitude,q-score,geometry,index_right,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode
18985,40.798405,-73.910843,24.001949,POINT (-73.910843 40.798405),858,86992,103.0,2583,19,None,145500,0,2,3.0,145399.293860,1,1008650,229901,1977,10454
18986,40.798481,-73.910065,13.967658,POINT (-73.910065 40.798481),1084,0,0.0,2583,19,07,213874,0,2,0.0,81598.263847,0,1009295,230090,0,10454
18987,40.798489,-73.910446,15.963887,POINT (-73.91044599999999 40.798489),1084,0,0.0,2583,19,07,213874,0,2,0.0,81598.263847,0,1009295,230090,0,10454
18989,40.798519,-73.911476,17.026796,POINT (-73.91147600000001 40.798519),1096,447145,377.5,2583,19,06,350500,0,2,4.0,342200.895984,1,1008388,230038,1999,10454
19070,40.798824,-73.911690,22.293566,POINT (-73.91169000000001 40.798824),1096,447145,377.5,2583,19,06,350500,0,2,4.0,342200.895984,1,1008388,230038,1999,10454


In [14]:
BX_Shape_joined.to_csv('BX_Geometry_Joined.csv')

In [15]:
BX_Shape_joined_2.to_csv('BX_Point_Joined.csv')